## Project 1.3 - Test Functions 

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (633291), Ida Wöstheinreich (628428)

In [91]:
using LinearAlgebra,Random,Printf,FFTW,JLD,Plots,LaTeXStrings,Colors
include("Ground_state.jl"); include("1.3_Evolution.jl")

simulate_2D

### Two important properties
- **The unitarity of integrators**\
    The exact evolution operator is a unitary operator, i.e. it satisfies 
    $$U_\text{ex}^\dagger U_\text{ex} =1.$$
    The unitarity property in the context of quantum mechanics is also often called $\textit{conservation of total probability}$, because this property is equivalent to the statement that the norm of states is conserved by the time evolution. 

- **The energy conservation of integrators**\
    The exact evolution operator satisfies
    $$U_\text{ex}^\dagger \hat{H}U_\text{ex} =\hat{H}$$
    This property implies that the expectation value of the energy does not depend on time, i.e. it is conserved.

In [92]:
function test_unitarity(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number , method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)] # length(m) == ndims(n) 
    function test_engine(shape::Tuple{Vararg{Int}})   
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)])
        P = plan_fft(ψ_0); P_inv = inv(P)
        integrator = choose_integrator(method; tol_cg, maxiters_cg, P, P_inv)
        norm_ψ_0 = norm(ψ_0) 
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        norm_ψ = norm(ψ)
        residue = abs(norm_ψ - norm_ψ_0) 
        @printf "residue = %.3e" residue 
    end
    print("\nConservation of the Norm of the states with the "); print(method); print(" method after a time step(τ = "); print(τ) 
    print("): \n");
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")   
end 
function test_energy_conservation(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number, method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)]
    function test_engine(shape::Tuple{Vararg{Int}}) 
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)]) 
        P = plan_fft(ψ_0); P_inv = inv(P)
        integrator = choose_integrator(method; tol_cg, maxiters_cg, P, P_inv) 
        E_0,_ = energy_expectation(ψ_0, μ, ϵ)
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        E,_= energy_expectation(ψ, μ, ϵ)
        residue = abs(E - E_0)
        @printf "residue = %.3e" residue
    end
    print("\nConservation of the energy expectation with the "); print(method); print(" method after a time step(τ = "); print(τ)
    print("): \n"); 
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    print("\nD=3: "); test_engine((N,N,N)); print("\n")   
end

test_energy_conservation (generic function with 3 methods)

Not all approximated integrators preserve unitarity and energy conservation exactly.
For example, the `Euler integrator` is neither unitary nor energy-preserving. The residues are slightly higher for higher dimensions.  
But both properties are recovered in the τ → 0 limit:

In [93]:
N = 10
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[1])
    test_energy_conservation(N, μ, ϵ, t, σ, method[1])
end


Conservation of the Norm of the states with the euler method after a time step(τ = 0.5): 
D=1: 

residue = 2.043e-01
D=2: 

residue = 1.472e-01
D=3: 

residue = 1.061e-01



Conservation of the energy expectation with the euler method after a time step(τ = 0.5): 
D=1: residue = 2.993e+00
D=2: residue = 1.555e+00
D=3: 

residue = 8.084e-01

Conservation of the Norm of the states with the euler method after a time step(τ = 0.05): 
D=1: residue = 1.342e-02
D=2: residue = 9.669e-03
D=3: residue = 6.969e-03

Conservation of the energy expectation with the euler method after a time step(τ = 0.05): 
D=1: residue = 2.993e-02
D=2: residue = 1.555e-02
D=3: residue = 8.084e-03

Conservation of the Norm of the states with the euler method after a time step(τ = 0.0005): 
D=1: residue = 2.204e-06
D=2: residue = 1.589e-06
D=3: residue = 1.145e-06

Conservation of the energy expectation with the euler method after a time step(τ = 0.0005): 
D=1: residue = 2.993e-06
D=2: residue = 1.555e-06
D=3: residue = 8.084e-07


But the `Crank-Nicolson` integrator is unitary and conserves the energy. 
The residues for these method are consistently of order $10^{-10}$ or smaller for both tests.
The size of the residues does not depend on the size of τ:

In [94]:
N = 10
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[2])
    test_energy_conservation(N, μ, ϵ, t, σ, method[2])
end


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.5): 
D=1: residue = 7.164e-16
D=2: 

residue = 1.225e-15
D=3: 

residue = 1.739e-14

Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.5): 
D=1: residue = 1.601e-14
D=2: residue = 7.589e-17
D=3: 

residue = 5.601e-15

Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 1.695e-15
D=2: residue = 1.493e-15
D=3: residue = 1.516e-14

Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 1.575e-15
D=2: residue = 1.542e-15
D=3: 

residue = 9.103e-15

Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 1.388e-17
D=2: residue = 2.042e-14
D=3: residue = 2.679e-15

Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 7.520e-16
D=2: residue = 3.036e-18
D=3: residue = 2.921e-16


The `Strang-splitting` integrator is unitary and but does not conserve the energy. 
The residue for the unitary test are consistently of order $10^{-11}$ or smaller and
their size does not depend on the size of τ.\
Energy conservation is recovered in the τ → 0 limit:

In [95]:
N = 10
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[3])
    test_energy_conservation(N, μ, ϵ, t, σ, method[3])
end


Conservation of the Norm of the states with the strang-splitting method after a time step(τ = 0.5): 
D=1: 

residue = 1.735e-18
D=2: 

residue = 8.674e-19
D=3: 

residue = 0.000e+00

Conservation of the energy expectation with the strang-splitting method after a time step(τ = 0.5): 
D=1: residue = 3.691e-05
D=2: residue = 1.868e-05
D=3: residue = 9.450e-06

Conservation of the Norm of the states with the strang-splitting method after a time step(τ = 0.05): 
D=1: residue = 1.735e-18
D=2: residue = 0.000e+00
D=3: residue = 8.674e-19

Conservation of the energy expectation with the strang-splitting method after a time step(τ = 0.05): 
D=1: residue = 1.442e-06
D=2: residue = 7.274e-07
D=3: residue = 3.668e-07

Conservation of the Norm of the states with the strang-splitting method after a time step(τ = 0.0005): 
D=1: residue = 0.000e+00
D=2: residue = 0.000e+00
D=3: residue = 0.000e+00

Conservation of the energy expectation with the strang-splitting method after a time step(τ = 0.0005): 
D=1: residue = 2.939e-12
D=2: residue = 1.516e-12
D=3: residue = 7.822e-13
